<a href="https://colab.research.google.com/github/TasnimAhmedEee/House-Price-Prediction/blob/master/optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example of Implementing Optuna for optimizing Neural Network

In [1]:
#importing libraries

#!pip install optuna
import os
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import numpy as np
from torchvision import datasets
from torchvision import transforms

In [2]:
#variables
DEVICE = torch.device("cuda")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = np.multiply(BATCHSIZE , 30) 
N_VALID_EXAMPLES = np.multiply(BATCHSIZE , 10) 

In [3]:
#Defining neural network
#Variables under "trial" object will be optimized by optuna

def define_model(trial):
    # Letting optuna to optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = np.multiply(28 , 28)
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [4]:
# Loading train and validation data

def get_mnist():
    # Loading FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

In [5]:
# defining objective for an optuna trial
def objective(trial):

    # Generating the model.
    model = define_model(trial).to(DEVICE)

    # Generating the optimizers. Variables under "trial" object will be optimized by optuna
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if np.multiply(batch_idx , BATCHSIZE) >= N_TRAIN_EXAMPLES:
                break
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data for faster epochs.
                if np.multiply(batch_idx , BATCHSIZE) >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)
        trial.report(accuracy, epoch)

        # Handling pruning based on the intermediate value (accuracy).
        # Pruned trials will not complete all epochs resulting in fast completion of all trials
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return accuracy

In [6]:
#The main function
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    
    #optuna is set to try 100 trials max
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-07-08 04:54:27,718] A new study created in memory with name: no-name-a092b5ec-fff5-4033-b4a7-e69fe5d88ebe
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)

[I 2021-07-08 04:54:34,885] Trial 0 finished with value: 0.69140625 and parameters: {'n_layers': 2, 'n_units_l0': 112, 'dropout_l0': 0.4678724546600941, 'n_units_l1': 13, 'dropout_l1': 0.46629364110086, 'optimizer': 'RMSprop', 'lr': 0.0007958239880843771}. Best is trial 0 with value: 0.69140625.
[I 2021-07-08 04:54:38,888] Trial 1 finished with value: 

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  48
  Number of complete trials:  52
Best trial:
  Value:  0.85390625
  Params: 
    n_layers: 1
    n_units_l0: 67
    dropout_l0: 0.2837784348441891
    optimizer: Adam
    lr: 0.0074902770836033665


In [7]:
# checking the best observations for objective value and their trials
optuna.visualization.plot_optimization_history(study)

In [8]:
df = study.trials_dataframe()
df.sort_values(by = 'value', ascending=False).head(5)

,number,value,datetime_start,datetime_complete,duration,params_dropout_l0,params_dropout_l1,params_dropout_l2,params_lr,params_n_layers,params_n_units_l0,params_n_units_l1,params_n_units_l2,params_optimizer,state
47,47,0.853906,2021-07-08 04:56:33.317654,2021-07-08 04:56:37.118042,0 days 00:00:03.800388,0.283778,NaN,NaN,0.007490,1,67,NaN,NaN,Adam,COMPLETE
92,92,0.848437,2021-07-08 04:58:06.615546,2021-07-08 04:58:10.439549,0 days 00:00:03.824003,0.231622,NaN,NaN,0.004035,1,108,NaN,NaN,Adam,COMPLETE
84,84,0.847656,2021-07-08 04:57:52.483930,2021-07-08 04:57:56.254526,0 days 00:00:03.770596,0.263516,NaN,NaN,0.004215,1,124,NaN,NaN,Adam,COMPLETE
83,83,0.847656,2021-07-08 04:57:48.708724,2021-07-08 04:57:52.482397,0 days 00:00:03.773673,0.281449,NaN,NaN,0.005050,1,120,NaN,NaN,Adam,COMPLETE
40,40,0.842187,2021-07-08 04:56:15.320229,2021-07-08 04:56:19.120501,0 days 00:00:03.800272,0.312099,NaN,NaN,0.003310,1,68,NaN,NaN,Adam,COMPLETE


In [9]:
#Parallel Coordinate plot for specific hyper-parameters to observe their impact on the objective value (classification accuracy) 
optuna.visualization.plot_parallel_coordinate(study, params=["lr", "optimizer", "n_layers"])

In [10]:
#Parallel Coordinate plot for all hyper-parameters to observe their trial range in Optuna
optuna.visualization.plot_parallel_coordinate(study)